In [12]:
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor
from io import StringIO
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

SCHEDVIEW = "./build/schedview/schedview"
PLATFORM = "./platforms/exynos5422LITTLE.json"

def call_cmpt(log, index):
    args = [SCHEDVIEW, "--platform", PLATFORM, log, "--config"]
    if index:
        args.append("--index")
    return subprocess.run(args, capture_output=True, text=True, check=True).stdout

def process_logs(logsdir, first_flag):
    files = sorted(os.listdir(logsdir))
    results = []
    first = first_flag
    for file in files:
        log = os.path.join(logsdir, file)
        results.append(call_cmpt(log, first))
        first = False

    print(results)
    return ''.join(results)


def process_logs(logsdir, log):
    """
    Process log files in a directory, compute duration results for PA, FFA, and CSF,
    and normalize the differences compared to PA.
    """
    # Define a bin size
    bin_size = 1000  # Example bin size, can be adjusted as needed
    
    # Get the list of files in the directory
    log_files = [os.path.join(logsdir, f) for f in os.listdir(logsdir) if os.path.isfile(os.path.join(logsdir, f))]

    print(log_files)
    
    # Initialize results
    bins_pa = []
    bins_ffa = []
    bins_csf = []
    normalized_differences = []
    
    for log_file in log_files:
        # Compute the PA version and duration
        pa_duration = compute_duration(log_file, "PA")
        bins_pa.append(pa_duration)
        
        # Compute the FFA version and duration
        ffa_duration = compute_duration(log_file, "FFA")
        bins_ffa.append(ffa_duration)
        
        # Compute the CSF version and duration
        csf_duration = compute_duration(log_file, "CSF")
        bins_csf.append(csf_duration)
        
        # Compute the difference compared to PA
        diff_ffa_pa = ffa_duration - pa_duration
        diff_csf_pa = csf_duration - pa_duration
        
        # Normalize the difference
        normalized_diff_ffa = diff_ffa_pa / pa_duration if pa_duration != 0 else 0
        normalized_diff_csf = diff_csf_pa / pa_duration if pa_duration != 0 else 0
        
        # Add to the result list
        normalized_differences.append({
            "log_file": log_file,
            "normalized_diff_ffa": normalized_diff_ffa,
            "normalized_diff_csf": normalized_diff_csf
        })
    
    # Example of placing durations in bins
    pa_bins = np.histogram(bins_pa, bins=np.arange(0, max(bins_pa) + bin_size, bin_size))
    ffa_bins = np.histogram(bins_ffa, bins=np.arange(0, max(bins_ffa) + bin_size, bin_size))
    csf_bins = np.histogram(bins_csf, bins=np.arange(0, max(bins_csf) + bin_size, bin_size))
    
    return {
        "pa_bins": pa_bins,
        "ffa_bins": ffa_bins,
        "csf_bins": csf_bins,
        "normalized_differences": normalized_differences
    }


def compute_duration(log_file, version):
    """
    Stub function to compute duration for a given log file and version.
    Replace with actual logic for parsing and extracting durations.
    """
    # Example placeholder logic
    durations = {
        "PA": 100,
        "FFA": 120,
        "CSF": 90
    }
    return durations.get(version, 0)

def compute(util):
    logsdirs = {
        "pa": f"data_umax0_9_logs_pa_no_delay/{util}/",
        "ffa": f"data_umax0_9_logs_ffa_no_delay/{util}/",
        "csf": f"data_umax0_9_logs_csf_no_delay/{util}/"
    }
    
    # Process logs concurrently
    with ThreadPoolExecutor() as executor:
        results = {key: executor.submit(process_logs, dir, True) for key, dir in logsdirs.items()}
    
    # Parse results into Polars dataframes
    dfs = {
        key: pl.read_csv(StringIO(future.result()), separator=";")
        for key, future in results.items()
    }
    
    for key in dfs:
        dfs[key] = dfs[key].with_columns(
            (pl.col("stop") - pl.col("start")).alias("duration")
        )

    print(dfs)

    return 
        
    # Prepare plots
    fig, axs = plt.subplots(1, 3, figsize=(10, 3))
    max_duration = max(
        dfs['pa']["duration"].max(), 
        dfs['ffa']["duration"].max(), 
        dfs['csf']["duration"].max()
    )
    bins = np.linspace(0, max_duration, 1001)
    
    histogram_data = {}
    
    for key, title in [("pa", "GRUB Power Aware"), ("ffa", "GRUB FFA"), ("csf", "GRUB CSF")]:
        # Compute histogram
        counts, bin_edges = np.histogram(dfs[key]["duration"].to_numpy(), bins=bins)
        histogram_data[key] = pd.DataFrame({
            "bin_start": bin_edges[:-1],  # Start of each bin
            "bin_end": bin_edges[1:],    # End of each bin
            "count": counts              # Count in the bin
        })
        
        # Optional: Plotting for visualization
        ax = axs[list(histogram_data.keys()).index(key)]
        ax.hist(dfs[key]["duration"].to_numpy(), bins=bins)
        ax.set_title(title)
        ax.set_xlim(-100, 3000)
        ax.set_ylim(0, 1250)
    
    #plt.tight_layout()
    #plt.show()
    
    # Save each histogram to CSV
    for key, data in dfs.items():
        data.write_csv(f"{key}_{util}_config_duration.csv")

In [13]:
compute("util_0_1")

['data_umax0_9_logs_ffa_no_delay/util_0_1/20.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/98.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/77.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/61.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/36.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/41.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/16.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/57.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/94.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/82.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/6.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/7.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/83.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/95.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/56.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/17.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/40.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/37.json', 'data_umax0_9_logs_ffa_no_delay/util_0_1/60.json', 'data_umax0_9_logs_ffa_no_delay/

TypeError: initial_value must be str or None, not dict

In [3]:
compute("util_0_2")

KeyboardInterrupt: 

In [ ]:
compute("util_0_3")

In [ ]:
compute("util_0_4")

In [ ]:
compute("util_0_5")

In [ ]:
compute("util_0_6")

In [ ]:
compute("util_0_7")

In [ ]:
compute("util_0_8")

In [ ]:
compute("util_0_9")

In [ ]:
compute("util_1_0")

In [ ]:
compute("util_1_1")

In [ ]:
compute("util_1_2")

In [ ]:
compute("util_1_3")

In [ ]:
compute("util_1_4")

In [ ]:
compute("util_1_5")

In [ ]:
compute("util_1_6")

In [ ]:
compute("util_1_7")

In [ ]:
compute("util_1_8")

In [ ]:
compute("util_1_9")

In [ ]:
compute("util_2_0")

In [ ]:
compute("util_2_1")

In [ ]:
compute("util_2_2")

In [ ]:
compute("util_2_3")

In [ ]:
compute("util_2_4")

In [ ]:
compute("util_2_5")

In [ ]:
compute("util_2_6")

In [ ]:
compute("util_2_7")

In [ ]:
compute("util_2_8")

In [ ]:
compute("util_2_9")

In [ ]:
compute("util_3_0")

In [ ]:
compute("util_3_1")

In [ ]:
compute("util_3_2")

In [ ]:
compute("util_3_3")

In [ ]:
compute("util_3_4")

In [ ]:
compute("util_3_5")

In [ ]:
compute("util_3_6")

In [ ]:
compute("util_3_7")

In [ ]:
compute("util_3_8")

In [ ]:
compute("util_3_9")

In [8]:
compute("util_4_0")

['active_cores;freq;start;stop\n4;1200.000000;0.000000;129.039132\n4;1000.000000;129.039132;23598.749500\n4;400.000000;23598.749500;23789.158706\n4;200.000000;23789.158706;24023.063113\n', '4;1000.000000;0.000000;310.028983\n4;800.000000;310.028983;24587.234826\n4;600.000000;24587.234826;25143.248574\n4;200.000000;25143.248574;25157.436430\n', '4;1300.000000;0.000000;27.640473\n4;1200.000000;27.640473;18834.758466\n4;1000.000000;18834.758466;24588.839678\n4;800.000000;24588.839678;25133.106373\n4;400.000000;25133.106373;25157.413055\n4;200.000000;25157.413055;25160.989863\n', '4;600.000000;0.000000;0.189773\n4;400.000000;0.189773;24800.255885\n4;200.000000;24800.255885;25008.151218\n', '4;1300.000000;0.000000;22.262530\n4;1200.000000;22.262530;21907.636660\n4;1000.000000;21907.636660;23859.635826\n4;200.000000;23859.635826;24492.472224\n', '4;1000.000000;0.000000;263.785510\n4;800.000000;263.785510;23517.862895\n4;400.000000;23517.862895;24009.848269\n4;200.000000;24009.848269;24947.55